In [1]:
import os
from glob import glob
import dill
import pathlib
import pandas as pd
import numpy as np
!pip3 install pydub
import pydub
!pip3 install python_speech_features
import python_speech_features
from utils import * 
import re

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=6ef357f331711018f2913f94a2906b8bd9036d1c078e98af1ce1d2db7ad923fa
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:

base_folder = pathlib.Path("/content/drive/My Drive/validated_clips/validated_clip/")
out_folder =  pathlib.Path("/content/drive/My Drive/validated_clips/preprocessed/")
outfiles = os.listdir(out_folder)
already_done = set()

for file in outfiles:
  thing = re.compile('[a-z]')
  already_done.add(file[:thing.search(file).start()])

folders = os.listdir(base_folder)
for folder in folders:
  if folder in already_done:
    print(f'skipping {folder}')
    continue
  print(folder)
  inner_folder = base_folder / folder
  globby = str(inner_folder)+"/*.mp3"
  downloaded_files = glob(globby)
  #print(downloaded_files)

  filepath_dict = {pathlib.Path(filepath).name : filepath for filepath in downloaded_files}
  print(filepath_dict)

  dev = pd.read_csv("/content/drive/My Drive/tsvs/dev.tsv", sep="\t")
  train = pd.read_csv("/content/drive/My Drive/tsvs/train.tsv", sep="\t")
  test = pd.read_csv("/content/drive/My Drive/tsvs/test.tsv", sep="\t")
  splits = {
      "dev": dev,
      "train": train, 
      "test": test
  }

  #only files in current folder
  for key in splits:
    print(key)
    print(splits[key].shape)
    splits[key]["path"] = splits[key]["path"].map(filepath_dict)
    splits[key] = splits[key][splits[key]['path'].notna()]
    print(splits[key].shape)

  #only ones with accents
  for key in splits:
    splits[key].dropna(axis=0, subset=["accent"], inplace=True)
  

  downloaded_audios_dict = {key : [pydub.AudioSegment.from_mp3(f) for f in splits[key]['path']] for key in splits}
  max_lengths = [np.max([len(sample) for sample in samples]) for samples in downloaded_audios_dict.values() if samples != []]
  if max_lengths == []:
    print("I cry")
    continue
  max_audio_length = np.max(max_lengths)
  padded_audio_dict = {key: [zero_pad_in_end(audio, max_audio_length) for audio in audios] for key, audios in downloaded_audios_dict.items()}

  labels_dict = {key: splits[key]['accent'].to_numpy() for key in splits}
  paths_dict ={key: [f for f in splits[key]['path']] for key in splits}
  mfccs_padded_dict = {key: np.array([extract_mfcc(audio) for audio in padded_audios]) for key, padded_audios in padded_audio_dict.items()}

  label_filename = folder+"label.pkl"
  paths_filename = folder+"paths.pkl"
  mfccs_padded_filename = folder+"mfccs_padded.pkl"

  label_filepath = out_folder / label_filename
  paths_filepath = out_folder / paths_filename
  mfccs_padded_filepath = out_folder / mfccs_padded_filename

  with open(label_filepath , "wb") as f:
    dill.dump(labels_dict, f)
  with open(paths_filepath , "wb") as f:
    dill.dump(paths_dict, f)
  with open(mfccs_padded_filepath , "wb") as f:
    dill.dump(mfccs_padded_dict, f)

skipping 103.000000
skipping 101.000000
skipping 100.000000
skipping 105.000000
skipping 104.000000
skipping 106.000000
skipping 1.000000
skipping 107.000000
skipping 102.000000
skipping 10.000000
skipping 114.000000
skipping 116.000000
skipping 11.000000
skipping 109.000000
skipping 115.000000
skipping 110.000000
skipping 111.000000
skipping 112.000000
skipping 113.000000
skipping 108.000000
skipping 120.000000
skipping 122.000000
skipping 117.000000
skipping 124.000000
skipping 119.000000
skipping 125.000000
skipping 121.000000
skipping 12.000000
skipping 123.000000
skipping 118.000000
skipping 127.000000
skipping 130.000000
skipping 129.000000
skipping 134.000000
skipping 128.000000
skipping 13.000000
skipping 133.000000
skipping 131.000000
skipping 126.000000
skipping 132.000000
skipping 142.000000
skipping 138.000000
skipping 141.000000
skipping 14.000000
skipping 135.000000
skipping 143.000000
skipping 140.000000
skipping 137.000000
skipping 136.000000
skipping 139.000000
skippin

In [5]:
print(len(already_done))

208


In [6]:
print(len(folders))

208
